<a href="https://colab.research.google.com/github/Sinem-B62/retail_demand_analysis/blob/main/data_prep_ipynbipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install -U gdown

import pandas as pd
import requests
import io


# Erstellen Sie die direkte Download-URL aus einer Datei-ID.
def make_drive_url(file_id):
    return f"https://drive.google.com/uc?id={file_id}"

# Hilfsfunktion zum Laden einer CSV-Datei über eine direkte URL
def load_csv_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Löst einen Fehler aus, wenn die Anfrage fehlschlägt.
    return pd.read_csv(io.StringIO(response.text))

# Verzeichnis der Datei-IDs zur besseren Übersichtlichkeit
file_ids = {
    "holiday_events": "1RMjSuqHXHTwAw_PGD5XVjhA3agaAGHDH",
    "items": "1ogMRixVhNY6XOJtIRtkRllyOyzw1nqya",
    "oil": "1Q59vk2v4WQ-Rpc9t2nqHcsZM3QWGFje_",
    "stores": "1Ei0MUXmNhmOcmrlPad8oklnFEDM95cDi",
    "train": "1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv",
    "transactions": "1PW5LnAEAiL43fI5CRDn_h6pgDG5rtBW_"
}

# Laden Sie jede CSV-Datei mit den Hilfsfunktionen.
df_holiday_events = load_csv_from_url(make_drive_url(file_ids["holiday_events"]))
df_items          = load_csv_from_url(make_drive_url(file_ids["items"]))
df_oil            = load_csv_from_url(make_drive_url(file_ids["oil"]))
df_stores         = load_csv_from_url(make_drive_url(file_ids["stores"]))
df_transactions   = load_csv_from_url(make_drive_url(file_ids["transactions"]))


In [43]:
import gdown

# Use our existing function to build the download URL
train_url = make_drive_url(file_ids["train"])

# Download the file using gdown
gdown.download(train_url, "train.csv", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv
From (redirected): https://drive.google.com/uc?id=1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv&confirm=t&uuid=4d3d883e-4e5a-4d95-9518-62b3b94363e3
To: /content/train.csv
100%|██████████| 5.00G/5.00G [01:07<00:00, 73.8MB/s]


'train.csv'

In [44]:
# Google Drive einbinden
from google.colab import drive
drive.mount('/content/drive')

# Speicherpfade im Drive definieren
save_dir = '/content/drive/MyDrive/Data Science_Sinem/Time Series'

save_path_pickle = f'{save_dir}/df_train_guayas.pkl'

# DataFrame speichern
df_train.to_pickle(save_path_pickle)

print("✅ Gespeichert als:")
print(save_path_pickle)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Gespeichert als:
/content/drive/MyDrive/Data Science_Sinem/Time Series/df_train_guayas.pkl


In [45]:
store_ids = df_stores[df_stores['state'] == 'Guayas']['store_nbr'].unique()
print("Anzahl Guayas-Stores:", len(store_ids))

Anzahl Guayas-Stores: 11


In [46]:
import pandas as pd

chunk_size = 10**6  # 1 million rows at a time
filtered_chunks = []

for chunk in pd.read_csv("train.csv", chunksize=chunk_size):
    chunk_filtered = chunk[chunk['store_nbr'].isin(store_ids)]
    filtered_chunks.append(chunk_filtered)
    del chunk  # Free up memory


/tmp/ipython-input-2678381120.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("train.csv", chunksize=chunk_size):


In [47]:
df_train = pd.concat(filtered_chunks, ignore_index=True)

In [48]:
df_train = df_train.sample(n=300_000).reset_index(drop=True)

## Top-3-Familien

In [49]:
items_per_family = df_items['family'].value_counts().head(3).reset_index()
items_per_family.columns = ['Family', 'Item Count']
top_3_family = items_per_family

print(top_3_family)


      Family  Item Count
0  GROCERY I        1334
1  BEVERAGES         613
2   CLEANING         446


In [50]:
item_ids = df_items[df_items['family'].isin(top_3_families['Family'].unique())]['item_nbr'].unique()

df_train = df_train[df_train['item_nbr'].isin(item_ids)]
df_train

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
1,101860103,2017-01-03,28,1153584,5.0,False
3,34036697,2014-10-28,51,1463812,25.0,False
4,55178189,2015-08-29,35,769312,5.0,False
5,60771258,2015-10-31,28,417835,1.0,False
8,103217170,2017-01-16,24,2015815,14.0,True
...,...,...,...,...,...,...
299991,72940911,2016-03-10,34,1464087,5.0,True
299992,106970738,2017-02-21,24,1397068,5.0,False
299995,28724874,2014-08-08,24,368260,19.0,False
299998,91540557,2016-09-20,28,1372326,5.0,False
